In [10]:
import backtesting
from backtesting import Strategy, Backtest
import pandas as pd
import keras
import numpy as np
import random
import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler

# Constants
# List of columns to scale
columns_to_scale = ['Open', 'High', 'Low','Close', 'Volume', 'RSI', 'ATR', 'MACD', 'MFI',
                    'EMA', 'SMA', 'OBV', 'GTrends_Interest', 'Sentiment_Bullish',
                    'Price_oil', 'Price_gold', 'Price_NASDAQ', 'Price_SP500', 'Price_NYSE',
                    'Interest_Rate', 'hash_rate', 'users']

# Define the function to create the dataset
def create_dataset(data, window_size, target_index):
    X, y = [], []
    for i in range(len(data) - window_size):
        X.append(data.iloc[i:(i + window_size)].values)  
        y.append(data.iloc[i + window_size, target_index])
    return np.array(X), np.array(y)

# Forecast Horizon
window_size = 3

# Target index ('Close')
close_index = 22

class CNN_Class_S1(Strategy):
    def init(self):
        self.trade_position = None
        self.days_since_position = 0

    def next(self):
        if self.data.Predictions > 0 and self.trade_position != 1:
            self.buy()
            self.trade_position = 1
            self.days_since_position = 0
        elif self.data.Predictions == 0 and self.trade_position != 0:
            self.sell()
            self.trade_position = 0
            self.days_since_position = 0

        # Close position if it's been S2 for a day
        if self.days_since_position == 1:
            if self.trade_position == 1:
                self.sell()
            else:
                self.buy()
            self.trade_position = None

        self.days_since_position += 1

class CNN_Class_S2(Strategy):
    def init(self):
        self.trade_position = None

    def next(self):
        # Long position logic
        if self.data.Predictions > 0 and self.trade_position != 1:
            if self.trade_position == 0:  
                self.buy()
            self.buy()  # S2 long position
            self.trade_position = 1
        # Short position logic
        elif self.data.Predictions == 0 and self.trade_position != 0:
            if self.trade_position == 1:  
                self.sell()
            self.sell() 
            self.trade_position = 0

# Full data #

In [11]:
seed_value = 123
tf.random.set_seed(seed_value)
np.random.seed(seed_value)
random.seed(seed_value)

# Load data
data = pd.read_csv('../../../Data/Final_df/BTC_FINAL_df_class.csv')

# Load model
full_data_model = keras.models.load_model('../../../Models/CNN-LSTM/CNN-LSTM-CLASS/best_CNNLSTM3DCLASS.keras')

# keep only the columns we need
col = ['Date', 'Open', 'High', 'Low', 'Close', 'Volume', 'Target']
backtest_data = data[col]
training_size = int(len(data) * 0.9)
backtest_data = backtest_data[training_size:]

# Split data only keep test data

# MAke the date the index
data.set_index('Date', inplace=True)

# Normalize the data
# Initialize the scaler
scaler = MinMaxScaler(feature_range=(0, 1))

# Scale the selected columns
data[columns_to_scale] = scaler.fit_transform(data[columns_to_scale])

# Train and test data
training_size = int(len(data) * 0.9)
test_data = data[training_size:]


# Create the dataset
X_test, y_test = create_dataset(test_data, window_size, close_index)

# Reshape the data
X_test = X_test.reshape((X_test.shape[0], window_size, X_test.shape[2], 1))


# Make predictions
predictions = full_data_model.predict(X_test)
test_pred = (predictions > 0.5).astype(int)

# create a df with 3 empty values (window size) and the predictions
test_pred = np.concatenate([np.zeros((window_size, 1)), test_pred])

# Add the predictions to the test data
backtest_data['Predictions'] = test_pred.astype(int)

# Erase the first 3 values
backtest_data = backtest_data[window_size:]

# set date as index
backtest_data.set_index('Date', inplace=True)
backtest_data.index = pd.to_datetime(backtest_data.index)

# S1 Max Strategy
bt1 = Backtest(backtest_data, CNN_Class_S1, cash=100000, commission=.0015, exclusive_orders=True, trade_on_close=True)
stats1 = bt1.run()


# Morethan S1 strategy
bt2 = Backtest(backtest_data, CNN_Class_S2, cash=100000, commission=.0015, exclusive_orders=True, trade_on_close=True)
stats2 = bt2.run()

# merge the stats
# Merge the stats
full_stats = pd.concat([stats1, stats2], axis=1, keys=['S1', 'S2'])
full_stats = full_stats.drop(['Start', 'End', 'Exposure Time [%]', 'Equity Final [$]', 'Equity Peak [$]',
                            'Sortino Ratio', 'Calmar Ratio', 'Max. Drawdown [%]', 'Avg. Drawdown [%]', 'Max. Drawdown Duration',
                            'Avg. Drawdown Duration', 'Max. Trade Duration', 'Avg. Trade Duration', 'SQN', '_strategy',
                            '_equity_curve', '_trades', 'Expectancy [%]','Best Trade [%]', 'Worst Trade [%]', 'Avg. Trade [%]',
                            'Profit Factor'],
                             axis=0)


full_stats.to_latex('cnn_lstm_full_stats.tex', float_format="%.2f")

full_stats

11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step


,S1,S2
Duration,340 days 00:00:00,340 days 00:00:00
Return [%],-40.969869,28.154931
Buy & Hold Return [%],150.974205,150.974205
Return (Ann.) [%],-43.119732,30.412093
Volatility (Ann.) [%],19.463424,48.034945
Sharpe Ratio,0.0,0.633124
# Trades,320,95
Win Rate [%],45.0,51.578947


# Bear Market #

## Bear Market 1 ##

In [12]:
seed_value = 1234
tf.random.set_seed(seed_value)
np.random.seed(seed_value)
random.seed(seed_value)

# Load data
bear_1 = pd.read_csv('../../../Data/Final_df/bear_market_1_classification.csv')

# Load model
bear_1_model = keras.models.load_model('../../../Models/CNN-LSTM/CNN-LSTM-CLASS/CNNLSTM-CLASS-bear_market_1.keras')

# keep only the columns we need
col = ['Date', 'Open', 'High', 'Low', 'Close', 'Volume', 'Target']
br_1_backtest_data = bear_1[col]
training_size = int(len(bear_1) * 0.9)
br_1_backtest_data = br_1_backtest_data[training_size:]

# Split data only keep test data

# MAke the date the index
bear_1.set_index('Date', inplace=True)


# Normalize the data
# Initialize the scaler
scaler = MinMaxScaler(feature_range=(0, 1))

# Scale the selected columns
bear_1[columns_to_scale] = scaler.fit_transform(bear_1[columns_to_scale])

# Train and test data
training_size = int(len(bear_1) * 0.9)
test_data = bear_1[training_size:]


# Create the dataset
X_test, y_test = create_dataset(test_data, window_size, close_index)

# Reshape the data
X_test = X_test.reshape((X_test.shape[0], window_size, X_test.shape[2], 1))

# Make predictions
br_1_predictions = bear_1_model.predict(X_test)
br_1_test_pred = (br_1_predictions > 0.5).astype(int)

# create a df with 3 empty values (window size) and the predictions
br_1_test_pred = np.concatenate([np.zeros((window_size, 1)), br_1_test_pred])

# Add the predictions to the test data
br_1_backtest_data['Predictions'] = br_1_test_pred.astype(int)

# Erase the first 3 values
br_1_backtest_data = br_1_backtest_data[window_size:]

# set date as index
br_1_backtest_data.set_index('Date', inplace=True)
br_1_backtest_data.index = pd.to_datetime(br_1_backtest_data.index)

# Buy and sell for 1 day
btS1 = Backtest(br_1_backtest_data, CNN_Class_S1, cash=100000, commission=.0015, exclusive_orders=True, trade_on_close=True)
statsS1 = btS1.run()


# Buy and keep S2 until other side prediction
bt = Backtest(br_1_backtest_data, CNN_Class_S2, cash=100000, commission=.0015, exclusive_orders=True, trade_on_close=True)
stats = bt.run()


# Merge the stats
full_stats_br1 = pd.concat([statsS1, stats], axis=1, keys=['S1', 'S2'])
full_stats_br1

1/2 ━━━━━━━━━━━━━━━━━━━━ 0s 72ms/step

/opt/anaconda3/lib/python3.11/site-packages/keras/src/saving/saving_lib.py:418: UserWarning: Skipping variable loading for optimizer 'adam', because it has 24 variables whereas the saved optimizer has 2 variables. 
  trackable.load_own_variables(weights_store.get(inner_path))


2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 247ms/step


,S1,S2
Start,2018-11-12 00:00:00,2018-11-12 00:00:00
End,2018-12-15 00:00:00,2018-12-15 00:00:00
Duration,33 days 00:00:00,33 days 00:00:00
Exposure Time [%],94.117647,94.117647
Equity Final [$],72835.770248,53101.830905
Equity Peak [$],103547.858663,100000.0
Return [%],-27.16423,-46.898169
Buy & Hold Return [%],-49.197543,-49.197543
Return (Ann.) [%],-96.671642,-99.888064
Volatility (Ann.) [%],4.371468,0.128412


## Bear Market 2 ##

In [13]:
seed_value = 1234
tf.random.set_seed(seed_value)
np.random.seed(seed_value)
random.seed(seed_value)

# Load data
data = pd.read_csv('../../../Data/Final_df/bear_market_2_classification.csv')

# Load model
full_data_model = keras.models.load_model('../../../Models/CNN-LSTM/CNN-LSTM-CLASS/CNNLSTM-CLASS-bear_market_2.keras')

# keep only the columns we need
col = ['Date', 'Open', 'High', 'Low', 'Close', 'Volume', 'Target']
backtest_data = data[col]
training_size = int(len(data) * 0.9)
backtest_data = backtest_data[training_size:]

# Split data only keep test data

# MAke the date the index
data.set_index('Date', inplace=True)

# Normalize the data
# Initialize the scaler
scaler = MinMaxScaler(feature_range=(0, 1))

# Scale the selected columns
data[columns_to_scale] = scaler.fit_transform(data[columns_to_scale])

# Train and test data
training_size = int(len(data) * 0.9)
test_data = data[training_size:]


# Create the dataset
X_test, y_test = create_dataset(test_data, window_size, close_index)

# Reshape the data
X_test = X_test.reshape((X_test.shape[0], window_size, X_test.shape[2], 1))


# Make predictions
predictions = full_data_model.predict(X_test)
test_pred = (predictions > 0.5).astype(int)

# create a df with 3 empty values (window size) and the predictions
test_pred = np.concatenate([np.zeros((window_size, 1)), test_pred])

# Add the predictions to the test data
backtest_data['Predictions'] = test_pred.astype(int)

# Erase the first 3 values
backtest_data = backtest_data[window_size:]

# set date as index
backtest_data.set_index('Date', inplace=True)
backtest_data.index = pd.to_datetime(backtest_data.index)

# S1 Max Strategy
bt1 = Backtest(backtest_data, CNN_Class_S1, cash=100000, commission=.0015, exclusive_orders=True, trade_on_close=True)
stats1 = bt1.run()


# Morethan S1 strategy
bt2 = Backtest(backtest_data, CNN_Class_S2, cash=100000, commission=.0015, exclusive_orders=True, trade_on_close=True)
stats2 = bt2.run()

# merge the stats
# Merge the stats
full_stats_br2 = pd.concat([stats1, stats2], axis=1, keys=['S1', 'S2'])
full_stats_br2

2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 70ms/step


/opt/anaconda3/lib/python3.11/site-packages/keras/src/saving/saving_lib.py:418: UserWarning: Skipping variable loading for optimizer 'adam', because it has 24 variables whereas the saved optimizer has 2 variables. 
  trackable.load_own_variables(weights_store.get(inner_path))


,S1,S2
Start,2022-10-17 00:00:00,2022-10-17 00:00:00
End,2022-11-21 00:00:00,2022-11-21 00:00:00
Duration,35 days 00:00:00,35 days 00:00:00
Exposure Time [%],94.444444,94.444444
Equity Final [$],80731.932383,116415.360493
Equity Peak [$],102428.615784,118059.567525
Return [%],-19.268068,16.41536
Buy & Hold Return [%],-19.249758,-19.249758
Return (Ann.) [%],-88.583234,366.950633
Volatility (Ann.) [%],8.662039,262.375179


In [14]:
# Merge the stats
cnn_lstm_bear_stats = pd.concat([full_stats_br1, full_stats_br2], axis=1, keys=['Bear_Market_1', 'Bear_Market_2'])


# Erase unecessary rows
cnn_lstm_bear_stats = cnn_lstm_bear_stats.drop(['Start', 'End', 'Exposure Time [%]', 'Equity Final [$]', 'Equity Peak [$]',
                            'Sortino Ratio', 'Calmar Ratio', 'Max. Drawdown [%]', 'Avg. Drawdown [%]', 'Max. Drawdown Duration',
                            'Avg. Drawdown Duration', 'Max. Trade Duration', 'Avg. Trade Duration', 'SQN', '_strategy',
                            '_equity_curve', '_trades', 'Expectancy [%]','Best Trade [%]', 'Worst Trade [%]', 'Avg. Trade [%]',
                            'Profit Factor'],
                             axis=0)

# Aggregate the stats
# Calculate the averages for each strategy
cnn_lstm_bear_stats['S1_Avg'] = (cnn_lstm_bear_stats[('Bear_Market_1', 'S1')] + cnn_lstm_bear_stats[('Bear_Market_2', 'S1')]) / 2
cnn_lstm_bear_stats['S2_Avg'] = (cnn_lstm_bear_stats[('Bear_Market_1', 'S2')] + cnn_lstm_bear_stats[('Bear_Market_2', 'S2')]) / 2

cnn_lstm_bear_stats.to_latex('cnn_lstm_bear_stats.tex', float_format="%.2f", multicolumn=True)

cnn_lstm_bear_stats

Bear_Market_1                       Bear_Market_2  \
                                     S1                S2                S1   
Duration               33 days 00:00:00  33 days 00:00:00  35 days 00:00:00   
Return [%]                    -27.16423        -46.898169        -19.268068   
Buy & Hold Return [%]        -49.197543        -49.197543        -19.249758   
Return (Ann.) [%]            -96.671642        -99.888064        -88.583234   
Volatility (Ann.) [%]          4.371468          0.128412          8.662039   
Sharpe Ratio                        0.0               0.0               0.0   
# Trades                             32                 1                34   
Win Rate [%]                      43.75               0.0         44.117647   

                                                   S1_Avg            S2_Avg  
                                     S2                                      
Duration               35 days 00:00:00  34 days 00:00:00  34 days 00:00:00  
Return [%]                     16.41536        -23.216149        -15.241404  
Buy & Hold Return [%]        -19.249758        -34.223651        -34.223651  
Return (Ann.) [%]            366.950633        -92.627438        133.531285  
Volatility (Ann.) [%]        262.375179          6.516754        131.251795  
Sharpe Ratio                   1.398572               0.0          0.699286  
# Trades                              3              33.0               2.0  
Win Rate [%]                  66.666667         43.933824         33.333333

# Bull Markets #

## Bull Market 1 ##

In [15]:
seed_value = 1234
tf.random.set_seed(seed_value)
np.random.seed(seed_value)
random.seed(seed_value)

# Load data
data = pd.read_csv('../../../Data/Final_df/bull_market_1_classification.csv')

# Load model
full_data_model = keras.models.load_model('../../../Models/CNN-LSTM/CNN-LSTM-CLASS/CNNLSTM-CLASS-bull_market_1.keras')

# keep only the columns we need
col = ['Date', 'Open', 'High', 'Low', 'Close', 'Volume', 'Target']
backtest_data = data[col]
training_size = int(len(data) * 0.9)
backtest_data = backtest_data[training_size:]

# Split data only keep test data

# MAke the date the index
data.set_index('Date', inplace=True)

# Normalize the data
# Initialize the scaler
scaler = MinMaxScaler(feature_range=(0, 1))

# Scale the selected columns
data[columns_to_scale] = scaler.fit_transform(data[columns_to_scale])

# Train and test data
training_size = int(len(data) * 0.9)
test_data = data[training_size:]


# Create the dataset
X_test, y_test = create_dataset(test_data, window_size, close_index)

# Reshape the data
X_test = X_test.reshape((X_test.shape[0], window_size, X_test.shape[2], 1))


# Make predictions
predictions = full_data_model.predict(X_test)
test_pred = (predictions > 0.5).astype(int)

# create a df with 3 empty values (window size) and the predictions
test_pred = np.concatenate([np.zeros((window_size, 1)), test_pred])

# Add the predictions to the test data
backtest_data['Predictions'] = test_pred.astype(int)

# Erase the first 3 values
backtest_data = backtest_data[window_size:]

# set date as index
backtest_data.set_index('Date', inplace=True)
backtest_data.index = pd.to_datetime(backtest_data.index)

# S1 Max Strategy
bt1 = Backtest(backtest_data, CNN_Class_S1, cash=100000, commission=.0015, exclusive_orders=True, trade_on_close=True)
stats1 = bt1.run()


# Morethan S1 strategy
bt2 = Backtest(backtest_data, CNN_Class_S2, cash=100000, commission=.0015, exclusive_orders=True, trade_on_close=True)
stats2 = bt2.run()


# merge the stats
# Merge the stats
full_stats_bl1 = pd.concat([stats1, stats2], axis=1, keys=['S1', 'S2'])
full_stats_bl1

4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step


/opt/anaconda3/lib/python3.11/site-packages/keras/src/saving/saving_lib.py:418: UserWarning: Skipping variable loading for optimizer 'adam', because it has 24 variables whereas the saved optimizer has 2 variables. 
  trackable.load_own_variables(weights_store.get(inner_path))


,S1,S2
Start,2017-09-04 00:00:00,2017-09-04 00:00:00
End,2017-12-17 00:00:00,2017-12-17 00:00:00
Duration,104 days 00:00:00,104 days 00:00:00
Exposure Time [%],98.113208,98.113208
Equity Final [$],47333.917751,432514.727849
Equity Peak [$],104708.561821,432514.727849
Return [%],-52.666082,332.514728
Buy & Hold Return [%],351.827192,351.827192
Return (Ann.) [%],-92.572587,16150.258474
Volatility (Ann.) [%],11.09045,23638.810579


## Bull Market 2 ##

In [16]:
seed_value = 1234
tf.random.set_seed(seed_value)
np.random.seed(seed_value)
random.seed(seed_value)

# Load data
data = pd.read_csv('../../../Data/Final_df/bull_market_2_classification.csv')

# Load model
full_data_model = keras.models.load_model('../../../Models/CNN-LSTM/CNN-LSTM-CLASS/CNNLSTM-CLASS-bull_market_2.keras')

# keep only the columns we need
col = ['Date', 'Open', 'High', 'Low', 'Close', 'Volume', 'Target']
backtest_data = data[col]
training_size = int(len(data) * 0.9)
backtest_data = backtest_data[training_size:]

# Split data only keep test data

# MAke the date the index
data.set_index('Date', inplace=True)

# Normalize the data
# Initialize the scaler
scaler = MinMaxScaler(feature_range=(0, 1))

# Scale the selected columns
data[columns_to_scale] = scaler.fit_transform(data[columns_to_scale])

# Train and test data
training_size = int(len(data) * 0.9)
test_data = data[training_size:]


# Create the dataset
X_test, y_test = create_dataset(test_data, window_size, close_index)

# Reshape the data
X_test = X_test.reshape((X_test.shape[0], window_size, X_test.shape[2], 1))


# Make predictions
predictions = full_data_model.predict(X_test)
test_pred = (predictions > 0.5).astype(int)

# create a df with 3 empty values (window size) and the predictions
test_pred = np.concatenate([np.zeros((window_size, 1)), test_pred])

# Add the predictions to the test data
backtest_data['Predictions'] = test_pred.astype(int)

# Erase the first 3 values
backtest_data = backtest_data[window_size:]

# set date as index
backtest_data.set_index('Date', inplace=True)
backtest_data.index = pd.to_datetime(backtest_data.index)

# S1 Max Strategy
bt1 = Backtest(backtest_data, CNN_Class_S1, cash=100000, commission=.0015, exclusive_orders=True, trade_on_close=True)
stats1 = bt1.run()


# Morethan S1 strategy
bt2 = Backtest(backtest_data, CNN_Class_S2, cash=100000, commission=.0015, exclusive_orders=True, trade_on_close=True)
stats2 = bt2.run()


# merge the stats
# Merge the stats
full_stats_bl2 = pd.concat([stats1, stats2], axis=1, keys=['S1', 'S2'])
full_stats_bl2

4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step


/opt/anaconda3/lib/python3.11/site-packages/keras/src/saving/saving_lib.py:418: UserWarning: Skipping variable loading for optimizer 'adam', because it has 24 variables whereas the saved optimizer has 2 variables. 
  trackable.load_own_variables(weights_store.get(inner_path))


,S1,S2
Start,2021-07-30 00:00:00,2021-07-30 00:00:00
End,2021-11-10 00:00:00,2021-11-10 00:00:00
Duration,103 days 00:00:00,103 days 00:00:00
Exposure Time [%],98.095238,98.095238
Equity Final [$],150364.832716,150566.387038
Equity Peak [$],150364.832716,151756.387038
Return [%],50.364833,50.566387
Buy & Hold Return [%],53.887508,53.887508
Return (Ann.) [%],318.519111,320.491311
Volatility (Ann.) [%],251.439429,259.632908


In [17]:
# Merge the stats
cnn_lstm_bull_stats = pd.concat([full_stats_bl1, full_stats_bl2], axis=1, keys=['Bull_Market_1', 'Bull_Market_2'])


# Erase unecessary rows
cnn_lstm_bull_stats = cnn_lstm_bull_stats.drop(['Start', 'End', 'Exposure Time [%]', 'Equity Final [$]', 'Equity Peak [$]',
                            'Sortino Ratio', 'Calmar Ratio', 'Max. Drawdown [%]', 'Avg. Drawdown [%]', 'Max. Drawdown Duration',
                            'Avg. Drawdown Duration', 'Max. Trade Duration', 'Avg. Trade Duration', 'SQN', '_strategy',
                            '_equity_curve', '_trades', 'Expectancy [%]','Best Trade [%]', 'Worst Trade [%]', 'Avg. Trade [%]',
                            'Profit Factor'],
                             axis=0)

# Aggregate the stats
# Calculate the averages for each strategy
cnn_lstm_bull_stats['S1_Avg'] = (cnn_lstm_bull_stats[('Bull_Market_1', 'S1')] + cnn_lstm_bull_stats[('Bull_Market_2', 'S1')]) / 2
cnn_lstm_bull_stats['S2_Avg'] = (cnn_lstm_bull_stats[('Bull_Market_1', 'S2')] + cnn_lstm_bull_stats[('Bull_Market_2', 'S2')]) / 2

cnn_lstm_bull_stats.to_latex('cnn_lstm_bull_stats.tex', float_format="%.2f", multicolumn=True)

cnn_lstm_bull_stats

Bull_Market_1                     \
                                      S1                 S2   
Duration               104 days 00:00:00  104 days 00:00:00   
Return [%]                    -52.666082         332.514728   
Buy & Hold Return [%]         351.827192         351.827192   
Return (Ann.) [%]             -92.572587       16150.258474   
Volatility (Ann.) [%]           11.09045       23638.810579   
Sharpe Ratio                         0.0           0.683209   
# Trades                             104                  1   
Win Rate [%]                        50.0              100.0   

                           Bull_Market_2                     \
                                      S1                 S2   
Duration               103 days 00:00:00  103 days 00:00:00   
Return [%]                     50.364833          50.566387   
Buy & Hold Return [%]          53.887508          53.887508   
Return (Ann.) [%]             318.519111         320.491311   
Volatility (Ann.) [%]         251.439429         259.632908   
Sharpe Ratio                    1.266783           1.234402   
# Trades                             103                  1   
Win Rate [%]                   55.339806              100.0   

                                  S1_Avg             S2_Avg  
                                                             
Duration               103 days 12:00:00  103 days 12:00:00  
Return [%]                     -1.150625         191.540557  
Buy & Hold Return [%]          202.85735          202.85735  
Return (Ann.) [%]             112.973262        8235.374892  
Volatility (Ann.) [%]          131.26494       11949.221743  
Sharpe Ratio                    0.633391           0.958806  
# Trades                           103.5                1.0  
Win Rate [%]                   52.669903              100.0